# Neighbour checks for quality control flags
Covers QC16-25

## Table of contents
[Neighbourhood functions](#neighbourhood-functions)  
[QC16 Daily neighbours (wet)](#QC16---Daily-neighbours-wet)  
[QC17 Hourly neighbours (wet)](#QC17---Hourly-neighbours-wet)  
[QC18 Daily neighbours (dry)](#QC18---Daily-neighbours-dry)  
[QC19 Hourly neighbours (dry)](#QC19---Hourly-neighbours-dry)  
[QC20 Monthly neighbours](#QC20---Monthly-neighbours)  
[QC21 Timing offset](#QC21---Timing-offset)  
[QC22 Pre-QC Affinity](#QC22---Pre-QC-Affinity)  
[QC23 Pre-QC Pearson](#QC23---Pre-QC-Pearson)  
[QC24 Daily factor](#QC24---Daily-factor)  
[QC25 Monthly factor](#QC25---Monthly-factor)  

See '3.3.4 Neighbouring gauge checks on large values' in Lewis et al. (2021)

In [1]:
import calendar
import datetime
import glob

import zipfile  ## used once for Intense format data
import pandas as pd  ## used once for Intense format data

import polars as pl
import numpy as np

import scipy.stats
import geopy.distance

In [2]:
TARGET_STATION_ID = "DE_02483"
DISTANCE_THRESHOLD = 50  # 50 km
OVERLAP_THRESHOLD = 365 * 3  # three years

## Data reading globals
GAUGE_DATA_PATH = "../data/gauge_data"
DATA_ROWS_TO_SKIP = 20  ## First 20 rows are metadata TODO: what if not?
UNIT_COL = "new_units"  ## There is an original_units col too TODO: think of way to do this for different data

MULTIPLYING_FACTORS = {"hourly": 24, "daily": 1}  ## compared to daily reference

In [3]:
def read_metadata(data_path):
    metadata = {}

    with open(data_path, "r") as f:
        while True:
            key, val = f.readline().strip().split(":", maxsplit=1)
            key = key.lower().replace(" ", "_")
            metadata[key.strip()] = val.strip()
            if key == "other":
                break
    return metadata

In [4]:
station_metadata = read_metadata(
    data_path=f"../data/gauge_data/{TARGET_STATION_ID}.txt"
)
station_metadata["start_datetime"] = datetime.datetime.strptime(
    station_metadata["start_datetime"], "%Y%m%d%H"
)
station_metadata["end_datetime"] = datetime.datetime.strptime(
    station_metadata["end_datetime"], "%Y%m%d%H"
)

In [5]:
def add_datetime_to_gauge_data(station_metadata, gauge_data, time_multiplying_factor):
    """
    Add datetime column to gauge data using metadata from that gauge.
    NOTE: Could maybe extend so can find metadata if not provided?
    """
    startdate = station_metadata["start_datetime"]
    enddate = station_metadata["end_datetime"]
    assert isinstance(
        startdate, datetime.datetime
    ), "Please convert start_ and end_datetime to datetime.datetime objects"

    date_interval = []
    delta_days = (enddate + datetime.timedelta(days=1) - startdate).days
    for i in range(delta_days * time_multiplying_factor):
        date_interval.append(startdate + datetime.timedelta(hours=i))

    ## add datetime column
    assert len(gauge_data) == len(date_interval)
    gauge_data = gauge_data.with_columns(
        time=pl.Series(date_interval)
    )  ## set time columns

    return gauge_data

In [6]:
rain_col = f"rain_{station_metadata['original_units']}"

In [7]:
## read in gauge data
target_gauge = pl.read_csv(
    f"../data/gauge_data/{TARGET_STATION_ID}.txt",
    skip_rows=20,
    schema_overrides={rain_col: pl.Float64},
)

In [8]:
target_gauge = add_datetime_to_gauge_data(
    station_metadata,
    target_gauge,
    time_multiplying_factor=MULTIPLYING_FACTORS["hourly"],
)
target_gauge = target_gauge.select(["time", rain_col])  ## Reorder (to look nice)

In [9]:
def replace_no_data_with_nan(data, no_data_value):
    return data.with_columns(
        pl.when(pl.col(rain_col) == no_data_value)
        .then(np.nan)
        .otherwise(pl.col(rain_col))
        .alias(rain_col)
    )

In [10]:
## make no data vals nans
target_gauge = replace_no_data_with_nan(
    target_gauge, no_data_value=int(station_metadata["no_data_value"])
)

In [11]:
target_gauge.head()

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0


# Neighbourhood functions
TODO: convert to Classes

### Part 1. Make or read summary metadata of stations

In [12]:
## Could work by checking if metadata already exists (or user can input)
all_gauge_data_paths = glob.glob(f"{GAUGE_DATA_PATH}/*.txt")
all_gauge_data_paths

['../data/gauge_data/DE_02718.txt',
 '../data/gauge_data/DE_00389.txt',
 '../data/gauge_data/DE_01300.txt',
 '../data/gauge_data/DE_00390.txt',
 '../data/gauge_data/DE_06264.txt',
 '../data/gauge_data/DE_02483.txt',
 '../data/gauge_data/DE_04488.txt',
 '../data/gauge_data/DE_03215.txt',
 '../data/gauge_data/DE_04313.txt',
 '../data/gauge_data/DE_00310.txt',
 '../data/gauge_data/DE_06303.txt']

In [13]:
all_station_metadata_list = []
for ind, file in enumerate(all_gauge_data_paths):
    one_station_metadata = read_metadata(data_path=file)
    all_station_metadata_list.append(one_station_metadata)

In [14]:
all_station_metadata = pl.from_dicts(all_station_metadata_list)
all_station_metadata = all_station_metadata.with_columns(
    pl.col("latitude").cast(pl.Float64),
    pl.col("longitude").cast(pl.Float64),
    (pl.col("start_datetime") + "00").str.strptime(pl.Datetime, "%Y%m%d%H%M"),
    (pl.col("end_datetime") + "00").str.strptime(pl.Datetime, "%Y%m%d%H%M"),
)
all_station_metadata.head()

station_id,country,original_station_number,original_station_name,path_to_original_data,latitude,longitude,start_datetime,end_datetime,elevation,number_of_records,percent_missing_data,original_timestep,new_timestep,original_units,new_units,time_zone,daylight_saving_info,no_data_value,resolution,other
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],str,str,str,str,str,str,str,str,str,str,str,str
"""DE_02718""","""Germany""","""02718""","""NA""","""B:/INTENSE data/Original data/…",51.288,8.7928,2006-01-01 00:00:00,2010-12-31 23:00:00,"""458m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_00389""","""Germany""","""00389""","""NA""","""B:/INTENSE data/Original data/…",51.0148,8.4318,2009-11-01 00:00:00,2010-12-31 23:00:00,"""436m""","""10224""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_01300""","""Germany""","""01300""","""NA""","""B:/INTENSE data/Original data/…",51.254,8.1565,2006-01-01 00:00:00,2010-12-31 23:00:00,"""351m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_00390""","""Germany""","""00390""","""NA""","""B:/INTENSE data/Original data/…",50.9837,8.3679,2006-01-01 00:00:00,2010-12-31 23:00:00,"""610m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_06264""","""Germany""","""06264""","""NA""","""B:/INTENSE data/Original data/…",51.4143,8.6498,2006-01-01 00:00:00,2010-12-31 23:00:00,"""457m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""


### Part 2. Compute distance from target station
TODO: What if the location data is in a different projection i.e. EPSG: 27700?  
TODO: Remove duplicates

In [15]:
def compute_distance_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    target_latlon = (
        target_station["latitude"].item(),
        target_station["longitude"].item(),
    )

    neighbour_distances = {}
    for other_station_id, other_lat, other_lon in metadata[
        ["station_id", "latitude", "longitude"]
    ].rows():

        neighbour_distances[other_station_id] = geopy.distance.geodesic(
            target_latlon, (other_lat, other_lon)
        ).kilometers
    return neighbour_distances

In [16]:
neighbours_distances = compute_distance_from_target_id(
    all_station_metadata, TARGET_STATION_ID
)
neighbours_distances

{'DE_02718': 24.361752991036987,
 'DE_00389': 18.844342416100808,
 'DE_01300': 24.64263807685722,
 'DE_00390': 23.462778369145934,
 'DE_06264': 28.343769567230837,
 'DE_02483': 0.0,
 'DE_04488': 15.929311137997068,
 'DE_03215': 15.710073576478786,
 'DE_04313': 35.397225659812456,
 'DE_00310': 13.134594190689885,
 'DE_06303': 13.96385857171046}

In [17]:
# ALTERNATIVE: do we want to avoid using geopy.distance and simply write a distance function?
# # ALTERNATIVE: maybe precompute a matrix of distances?
# all_station_dist_mtx = scipy.spatial.distance.cdist(all_station_metadata[['latitude', 'longitude']].rows(),
#                                         all_station_metadata[['latitude', 'longitude']].rows(),
#                                         metric=lambda pnt1, pnt2: geopy.distance.geodesic(pnt1, pnt2).kilometers)

In [18]:
neighbours_distances_df = pl.DataFrame(
    {
        "station_id": neighbours_distances.keys(),
        "distances": neighbours_distances.values(),
    }
)
neighbours_distances_df

station_id,distances
str,f64
"""DE_02718""",24.361753
"""DE_00389""",18.844342
"""DE_01300""",24.642638
"""DE_00390""",23.462778
"""DE_06264""",28.34377
…,…
"""DE_04488""",15.929311
"""DE_03215""",15.710074
"""DE_04313""",35.397226


In [19]:
## Subset based on 50 km
close_neighbours = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) & (pl.col("distances") != 0)
)

## closest 10
close_neighbours.sort("distances")[:10]

station_id,distances
str,f64
"""DE_00310""",13.134594
"""DE_06303""",13.963859
"""DE_03215""",15.710074
"""DE_04488""",15.929311
"""DE_00389""",18.844342
"""DE_00390""",23.462778
"""DE_02718""",24.361753
"""DE_01300""",24.642638
"""DE_06264""",28.34377


### Part 3. Compute the temporal overlap

In [20]:
def compute_overlap_days(start_1, end_1, start_2, end_2):
    ## TODO: add cast to datetime functionality/checks
    ## compute overlap
    overlap_start = max(start_1, start_2)
    overlap_end = min(end_1, end_2)

    overlap_days = max(0, (overlap_end - overlap_start).days)

    return overlap_days

In [21]:
def compute_overlap_days_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    start_1, end_1 = (
        target_station["start_datetime"].item(),
        target_station["end_datetime"].item(),
    )

    neighbour_overlap_days = {}
    for other_station_id, start_2, end_2 in metadata[
        ["station_id", "start_datetime", "end_datetime"]
    ].rows():
        if target_id == other_station_id:
            continue

        neighbour_overlap_days[other_station_id] = compute_overlap_days(
            start_1, end_1, start_2, end_2
        )
    return neighbour_overlap_days

In [22]:
neighbour_overlap_days = compute_overlap_days_from_target_id(
    all_station_metadata, TARGET_STATION_ID
)
neighbour_overlap_days

{'DE_02718': 1825,
 'DE_00389': 425,
 'DE_01300': 1825,
 'DE_00390': 1825,
 'DE_06264': 1825,
 'DE_04488': 1613,
 'DE_03215': 1309,
 'DE_04313': 1825,
 'DE_00310': 1825,
 'DE_06303': 1825}

#### Subset based on 3 years

In [23]:
neighbour_overlap_days_df = pl.DataFrame(
    {
        "station_id": neighbour_overlap_days.keys(),
        "overlap_days": neighbour_overlap_days.values(),
    }
)
neighbour_overlap_days_df

station_id,overlap_days
str,i64
"""DE_02718""",1825
"""DE_00389""",425
"""DE_01300""",1825
"""DE_00390""",1825
"""DE_06264""",1825
"""DE_04488""",1613
"""DE_03215""",1309
"""DE_04313""",1825
"""DE_00310""",1825


In [24]:
neighbour_overlap_days_df.filter(pl.col("overlap_days") >= OVERLAP_THRESHOLD)

station_id,overlap_days
str,i64
"""DE_02718""",1825
"""DE_01300""",1825
"""DE_00390""",1825
"""DE_06264""",1825
"""DE_04488""",1613
"""DE_03215""",1309
"""DE_04313""",1825
"""DE_00310""",1825
"""DE_06303""",1825


## Part 4. Bring together to get neighbours both close and overlapping

In [25]:
num_closest_gauges = 10  ## based on IntenseQC

In [26]:
## Subset based on 50 km
close_neighbour_ids = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) & (pl.col("distances") != 0)
)
## closest 10 values
closest_neighbour_ids = close_neighbour_ids.sort("distances")[:num_closest_gauges][
    "station_id"
].to_list()

## Subset based on 3 years
overlapping_neighbour_ids = neighbour_overlap_days_df.filter(
    pl.col("overlap_days") >= OVERLAP_THRESHOLD
)["station_id"].to_list()

In [27]:
all_neighbour_ids = set(overlapping_neighbour_ids).intersection(
    set(closest_neighbour_ids)
)
all_neighbour_ids

{'DE_00310',
 'DE_00390',
 'DE_01300',
 'DE_02718',
 'DE_03215',
 'DE_04313',
 'DE_04488',
 'DE_06264',
 'DE_06303'}

In [28]:
all_neighbour_ids_paths = {}
for id in all_neighbour_ids:
    ids_path = glob.glob(f"{GAUGE_DATA_PATH}/*{id}.txt")
    assert len(ids_path) == 1, f"There are {len(ids_path)} data files for {id}"
    all_neighbour_ids_paths[id] = ids_path[0]

In [29]:
all_neighbour_ids_paths

{'DE_04313': '../data/gauge_data/DE_04313.txt',
 'DE_04488': '../data/gauge_data/DE_04488.txt',
 'DE_02718': '../data/gauge_data/DE_02718.txt',
 'DE_06303': '../data/gauge_data/DE_06303.txt',
 'DE_00310': '../data/gauge_data/DE_00310.txt',
 'DE_00390': '../data/gauge_data/DE_00390.txt',
 'DE_03215': '../data/gauge_data/DE_03215.txt',
 'DE_01300': '../data/gauge_data/DE_01300.txt',
 'DE_06264': '../data/gauge_data/DE_06264.txt'}

## Part 5. Get neighbouring GDSR gauge by ID (an example)

In [30]:
def get_neighbouring_gdsr_data(neighbour_gdsr_id, time_multiplying_factor):
    data_path = all_neighbour_ids_paths[neighbour_gdsr_id]
    station_metadata = all_station_metadata.filter(
        pl.col("station_id") == neighbour_gdsr_id
    )
    assert (
        len(station_metadata) == 1
    ), f"There are {len(station_metadata)} metadata values for {neighbour_gdsr_id}. Investigate because there should only be one"
    station_metadata = station_metadata.to_dicts()[0]  ## convert df to a dict

    ## Read in gauge data
    units = station_metadata[UNIT_COL]
    rain_col = f"rain_{units}"
    gauge_data = pl.read_csv(
        data_path, skip_rows=DATA_ROWS_TO_SKIP, schema_overrides={rain_col: pl.Float64}
    )

    ## make datetime column
    gauge_data_w_dates = add_datetime_to_gauge_data(
        station_metadata, gauge_data, time_multiplying_factor=time_multiplying_factor
    )
    gauge_data_w_dates = gauge_data_w_dates.select(
        ["time", rain_col]
    )  ## Reorder (to look nice)

    return gauge_data_w_dates

In [31]:
get_neighbouring_gdsr_data(
    neighbour_gdsr_id="DE_06264", time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
)

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,0.1
2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


## Part 6. Get neighbouring GPCC gauge by ID (an example)

#### Note:
In the original methodology, GPCC is extracted on the fly

Hence, this needs to be refactored

In [32]:
# TODO: check whether GPCC name is same ID as GDSR i.e. gauge_data ID

In [33]:
existing_gpcc_daily_paths = {}
existing_gpcc_monthly_paths = {}
for neighbour_id in all_neighbour_ids_paths.keys():
    gpcc_id = neighbour_id.split("DE_")[1].lstrip("0")
    existing_gpcc_daily_paths[neighbour_id] = glob.glob(
        f"../data/GPCC/tw_{gpcc_id}.zip"
    )
    existing_gpcc_monthly_paths[neighbour_id] = glob.glob(
        f"../data/GPCC/mw_{gpcc_id}.zip"
    )

In [34]:
gpcc_id_to_use = "DE_00310"
gpcc_id = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303
example_dat_path = existing_gpcc_daily_paths[gpcc_id_to_use][0]
f = zipfile.ZipFile(example_dat_path).open(f"tw_{gpcc_id}.dat")
example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

## drop unnecessary columns
example_gpcc = example_gpcc.drop([str(i) for i in range(4, 16)])

## make datetime column (apparently it's 7am-7pm)
example_gpcc = example_gpcc.with_columns(
    pl.datetime(pl.col("2"), pl.col("1"), pl.col("0"), 7).alias("time")
).drop(["0", "1", "2"])

## rename and reorder
example_gpcc = example_gpcc.rename({"3": rain_col})
example_gpcc = example_gpcc.select(["time", rain_col])  ## Reorder (to look nice)

example_gpcc

time,rain_mm
datetime[μs],f64
1951-01-01 07:00:00,5.2
1951-01-02 07:00:00,10.3
1951-01-03 07:00:00,0.0
1951-01-04 07:00:00,3.7
1951-01-05 07:00:00,3.5
…,…
2018-12-27 07:00:00,0.0
2018-12-28 07:00:00,0.2
2018-12-29 07:00:00,6.9


In [35]:
## resample into daily (also round to 1 decimal place) TODO: check offset='7h' part
target_gauge_daily = (
    target_gauge.group_by_dynamic("time", every="1d", offset="7h")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).mean().round(1).alias(rain_col),
        ]
    )
    .filter(pl.col("n_hours") == 24)
    .drop("n_hours")
)  # Ensure at least 24 data points
target_gauge_daily

time,rain_mm
datetime[μs],f64
2006-01-01 07:00:00,0.2
2006-01-02 07:00:00,0.0
2006-01-03 07:00:00,0.0
2006-01-04 07:00:00,0.0
2006-01-05 07:00:00,0.0
…,…
2010-12-26 07:00:00,0.2
2010-12-27 07:00:00,0.0
2010-12-28 07:00:00,0.0


In [36]:
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_310
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


## Step 7 Compute factor, affinity index and correlation 

In [37]:
a = np.around(joined_gauges_gpcc.filter(pl.col(rain_col) >= 0.1).min()[rain_col], 1)[0]
b = np.around(
    joined_gauges_gpcc.filter(pl.col(f"{rain_col}_GPCC_{gpcc_id}") >= 0.1).min()[
        f"{rain_col}_GPCC_{gpcc_id}"
    ],
    1,
)[0]
p = max(a, b, 0.1)
print(a, b, p)

0.1 0.1 0.1


In [38]:
joined_gauges_gpcc_duplicates = joined_gauges_gpcc.with_columns(
    pl.when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id}") > p))
    .then(1)
    .when(
        (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id}") == p),
    )
    .then(1)
    .when(
        (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id}") > p),
    )
    .then(0)
    .when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id}") == p))
    .then(0)
    .otherwise(np.nan)
    .alias("duplicate")
)

joined_gauges_gpcc = joined_gauges_gpcc.with_columns(
    pl.when((pl.col(rain_col) > 0) & (pl.col(f"{rain_col}_GPCC_{gpcc_id}") > 0))
    .then(pl.col(rain_col) / pl.col(f"{rain_col}_GPCC_{gpcc_id}"))
    .otherwise(np.nan)
    .alias("factor")
)
joined_gauges_gpcc

time,rain_mm,rain_mm_GPCC_310,factor
datetime[μs],f64,f64,f64
2006-01-01 07:00:00,0.2,0.2,1.0
2006-01-02 07:00:00,0.0,0.0,NaN
2006-01-03 07:00:00,0.0,0.0,NaN
2006-01-04 07:00:00,0.0,0.0,NaN
2006-01-05 07:00:00,0.0,0.0,NaN
…,…,…,…
2010-12-26 07:00:00,0.2,2.9,0.068966
2010-12-27 07:00:00,0.0,0.2,NaN
2010-12-28 07:00:00,0.0,0.0,NaN


In [39]:
match = (
    joined_gauges_gpcc_duplicates["duplicate"]
    .value_counts()
    .filter(pl.col("duplicate") == 1)["count"]
    .item()
)
diff = (
    joined_gauges_gpcc_duplicates["duplicate"]
    .value_counts()
    .filter(pl.col("duplicate") == 0)["count"]
    .item()
)
perc = match / (match + diff)
p_corr = np.corrcoef(
    joined_gauges_gpcc[rain_col], joined_gauges_gpcc[f"{rain_col}_GPCC_{gpcc_id}"]
)[0, 1]
f_mean = joined_gauges_gpcc["factor"].drop_nans().mean()
print(f"diff: {diff}, match:{match}")
print(f"affinity: {perc}, p_corr: {p_corr}, f_mean: {f_mean}")

diff: 157, match:540
affinity: 0.7747489239598279, p_corr: 0.011471497096547798, f_mean: 0.17114986225908063


## Part 8 Compare target with neighbour (hourly and daily)
- For hourly data, the data is first converted to daily to do comparison

_Output:_ df with long list of neighbour columns and flags

Works by computing differences from target and each of its neighbours then collates all those differences and associated difference flags into a single flag/column that describes how similar target is from neighbours


In [40]:
gpcc_id_to_use = "DE_00310"
gpcc_id = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303

In [41]:
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_310
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


## Part 8.1 Wet neighbours
- This is normalised difference
TODO: Problem that there is not enough data which is wetter in the target than in the GPCC neighbour

In [42]:
def normalise_data(data):
    return (data - data.min()) / (data.max() - data.min())

In [43]:
joined_gauges_gpcc_normalised_diff = joined_gauges_gpcc.with_columns(
    # get normalised difference between target and neighbour
    rain_mm_normalised_diff=normalise_data(pl.col(f"{rain_col}"))
    - normalise_data(pl.col(f"{rain_col}_GPCC_{gpcc_id}"))
)

In [44]:
WET_THRESHOLD = 0.3  # default in original methodology is 1
joined_gauges_gpcc_normalised_diff_filtered = joined_gauges_gpcc_normalised_diff.filter(
    (pl.col(f"{rain_col}") >= WET_THRESHOLD)
    & (pl.col(f"{rain_col}").is_finite())
    & (pl.col(f"{rain_col}_GPCC_{gpcc_id}").is_finite())
    & (pl.col(f"{rain_col}_normalised_diff") > 0.0)
)

In [45]:
joined_gauges_gpcc_normalised_diff_filtered

time,rain_mm,rain_mm_GPCC_310,rain_mm_normalised_diff
datetime[μs],f64,f64,f64
2006-04-25 07:00:00,0.3,0.0,0.000941
2006-05-13 07:00:00,0.3,0.0,0.000941
2006-12-23 07:00:00,112.5,0.0,0.352886
2006-12-24 07:00:00,318.8,0.0,1.0
2007-03-16 07:00:00,3.1,0.0,0.009724
…,…,…,…
2009-11-06 07:00:00,9.8,0.9,0.015486
2009-11-18 07:00:00,1.2,0.0,0.003764
2009-11-19 07:00:00,0.8,0.0,0.002509


In [46]:
if not len(joined_gauges_gpcc_normalised_diff_filtered) >= 30:
    print(
        "Original methodology needs there to be at least 30 values to fit exponential function"
    )

Original methodology needs there to be at least 30 values to fit exponential function


In [47]:
expon_params = scipy.stats.expon.fit(
    joined_gauges_gpcc_normalised_diff_filtered[f"{rain_col}_normalised_diff"]
)

In [48]:
# Calculate thresholds at key percentiles of fitted distribution
q95 = scipy.stats.expon.ppf(0.95, expon_params[0], expon_params[1])
q99 = scipy.stats.expon.ppf(0.99, expon_params[0], expon_params[1])
q999 = scipy.stats.expon.ppf(0.999, expon_params[0], expon_params[1])

q95, q99, q999

(np.float64(0.3444910378117832),
 np.float64(0.5294660929815043),
 np.float64(0.7941055682411567))

In [49]:
## Assign flags
joined_gauges_gpcc_normalised_wet_flags = (
    joined_gauges_gpcc_normalised_diff.with_columns(
        pl.when(
            (pl.col(rain_col) >= 1.0) & (pl.col(f"{rain_col}_normalised_diff") <= q95)
        )
        .then(0)
        .when(
            (pl.col(rain_col) >= 1.0)
            & (pl.col(f"{rain_col}_normalised_diff") > q95)
            & (pl.col(f"{rain_col}_normalised_diff") <= q99),
        )
        .then(1)
        .when(
            (pl.col(rain_col) >= 1.0)
            & (pl.col(f"{rain_col}_normalised_diff") > q99)
            & (pl.col(f"{rain_col}_normalised_diff") <= q999),
        )
        .then(2)
        .when((pl.col(rain_col) >= 1.0) & (pl.col(f"{rain_col}_normalised_diff") > q95))
        .then(3)
        .otherwise(0)
        .alias("wet_flags")
    )
)

In [50]:
joined_gauges_gpcc_normalised_wet_flags["wet_flags"].value_counts()

wet_flags,count
i32,u32
2,1
3,1
1,1
0,1757


## Part 8.2 Dry neighbours

In [51]:
DRY_PERIOD_DAYS = 15
gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id}"

In [52]:
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_310
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


In [53]:
## TODO: This might use preexisting code, as this code is essentially looking for dry periods
def convert_to_dry_spell(rain_daily, dry_period_days, col_name):
    ## Convert to polars dataframe to use 'with_columns'
    if not isinstance(rain_daily, pl.DataFrame):
        rain_daily = rain_daily.to_frame()

    ## make dry day column
    rain_daily_dry_day = add_dry_day_col(rain_daily, col_name)
    rain_daily_dry_day = rain_daily_dry_day.with_columns(
        dry_spell_fraction=pl.col("dry_day").rolling_sum(
            window_size=dry_period_days, min_samples=dry_period_days
        )
        / dry_period_days
    )
    return rain_daily_dry_day["dry_spell_fraction"]


def add_dry_day_col(rain_daily, col_name):
    rain_daily_dry_day = rain_daily.with_columns(
        (pl.col(col_name) == 0).cast(pl.Int8()).alias("dry_day")
    )
    return rain_daily_dry_day

In [54]:
joined_gauges_gpcc_dry_spell = joined_gauges_gpcc.with_columns(
    pl.col(rain_col)
    .map_batches(
        lambda row: convert_to_dry_spell(row, DRY_PERIOD_DAYS, col_name=rain_col)
    )
    .alias("dry_spell_fraction"),
    pl.col(gpcc_col_name)
    .map_batches(
        lambda row: convert_to_dry_spell(row, DRY_PERIOD_DAYS, col_name=gpcc_col_name)
    )
    .alias(f"dry_spell_fraction_{gpcc_id}"),
)

In [55]:
joined_gauges_gpcc_dry_spell.head()

time,rain_mm,rain_mm_GPCC_310,dry_spell_fraction,dry_spell_fraction_310
datetime[μs],f64,f64,f64,f64
2006-01-01 07:00:00,0.2,0.2,null,null
2006-01-02 07:00:00,0.0,0.0,null,null
2006-01-03 07:00:00,0.0,0.0,null,null
2006-01-04 07:00:00,0.0,0.0,null,null
2006-01-05 07:00:00,0.0,0.0,null,null


In [56]:
## TODO: why have this part?
## consider only whether dry 15-day periods are corrobated as dry by neighbours
## dry flag works on the basis of fraction of dry days within 15-day
## check based on whether 0, 1, 2 or >= 3 wet days are recorded at the neighbour when the target is dry over the 15-day period
## truncating these fractions to 2 dp below and manipulating equalities to work with these fractions, but could work in days not fractions if change the convertToDrySpell function
dry_period = DRY_PERIOD_DAYS
fraction_drydays = {}
for d in range(1, 3 + 1):
    fraction_drydays[str(d)] = np.trunc((1.0 - (float(d) / dry_period)) * 10**2) / (
        10**2
    )

fraction_drydays  # moving window, so 1 = all dry, 0 = all wet

{'1': np.float64(0.93), '2': np.float64(0.86), '3': np.float64(0.8)}

In [57]:
## Assign flags
joined_gauges_gpcc_dry_flags = joined_gauges_gpcc_dry_spell.with_columns(
    pl.when(
        (pl.col("dry_spell_fraction") == 1.0)
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") == 1.0)
    )
    .then(0)
    .when(
        (pl.col("dry_spell_fraction") == 1.0)
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") < 1.0)
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["1"]),
    )
    .then(1)
    .when(
        (pl.col("dry_spell_fraction") == 1.0)
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["1"])
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["2"]),
    )
    .then(2)
    .when(
        (pl.col("dry_spell_fraction") == 1.0)
        & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["2"])
    )
    .then(3)
    .otherwise(0)
    .alias("dry_flags")
)

In [58]:
joined_gauges_gpcc_dry_flags["dry_flags"].value_counts()

dry_flags,count
i32,u32
1,1
2,4
0,1629
3,126


## Part 9 Compare target with neighbour (monthly) 
- Works differently from hourly and daily
- original methodology uses bfill which uses the next available value to fillNA

In [59]:
gpcc_id_to_use = "DE_03215"
gpcc_id = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303

In [60]:
## TODO: get gpcc monthly
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_3215
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


In [61]:
gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id}"

In [62]:
## resample into monthly
joined_gauges_gpcc_monthly = joined_gauges_gpcc.group_by_dynamic(
    "time", every="1mo", offset="7h"
).agg(
    [
        pl.col(rain_col).mean().round(1).alias(rain_col),
        pl.col(gpcc_col_name).mean().round(1).alias(gpcc_col_name),
    ]
)  # TODO: Ensure at least 24*n_days_in_month data points

In [63]:
joined_gauges_gpcc_monthly.head()

time,rain_mm,rain_mm_GPCC_3215
datetime[μs],f64,f64
2006-01-01 07:00:00,0.1,1.1
2006-02-01 07:00:00,0.3,3.0
2006-03-01 07:00:00,0.4,3.6
2006-04-01 07:00:00,0.3,2.3
2006-05-01 07:00:00,0.6,4.6


In [64]:
joined_gauges_gpcc_monthly_perc_diff = joined_gauges_gpcc_monthly.with_columns(
    perc_diff=((pl.col(rain_col) - pl.col(gpcc_col_name)) * 100.0)
    / pl.col(gpcc_col_name),
    factor_diff=pl.col(rain_col) / pl.col(gpcc_col_name),
)

In [65]:
joined_gauges_gpcc_monthly_flags = joined_gauges_gpcc_monthly_perc_diff.with_columns(
    pl.when(pl.col("perc_diff") <= -100.0)
    .then(-3)
    .when((pl.col("perc_diff") <= -50.0) & (pl.col("perc_diff") > -100.0))
    .then(-2)
    .when((pl.col("perc_diff") <= -25.0) & (pl.col("perc_diff") > -50.0))
    .then(-1)
    .when((pl.col("perc_diff") < 25.0) & (pl.col("perc_diff") > -25.0))
    .then(0)
    .when((pl.col("perc_diff") >= 25.0) & (pl.col("perc_diff") < 50.0))
    .then(1)
    .when((pl.col("perc_diff") >= 50.0) & (pl.col("perc_diff") < 100.0))
    .then(2)
    .when(pl.col("perc_diff") >= 100.0)
    .then(3)
    .otherwise(None)
    .alias("precip_flags"),
    pl.when((pl.col("factor_diff") < 11) & (pl.col("factor_diff") > 9))
    .then(1)
    .when((pl.col("factor_diff") < 26) & (pl.col("factor_diff") > 24))
    .then(2)
    .when((pl.col("factor_diff") < 3) & (pl.col("factor_diff") > 2))
    .then(3)
    .when((pl.col("factor_diff") > 1 / 11) & (pl.col("factor_diff") < 1 / 9))
    .then(4)
    .when((pl.col("factor_diff") > 1 / 26) & (pl.col("factor_diff") < 1 / 24))
    .then(5)
    .when((pl.col("factor_diff") > 1 / 3) & (pl.col("factor_diff") < 1 / 2))
    .then(6)
    .otherwise(0)
    .alias("factor_flags"),
)

In [66]:
joined_gauges_gpcc_monthly_flags["precip_flags"].value_counts()

precip_flags,count
i32,u32
-3,6
-2,52
3,2


In [67]:
joined_gauges_gpcc_monthly_flags["factor_flags"].value_counts()

factor_flags,count
i32,u32
4,5
0,55


In [68]:
## Set up NaN
joined_gauges_gpcc_monthly_flags = joined_gauges_gpcc_monthly_flags.with_columns(
    pl.when(pl.col(rain_col).is_nan())
    .then(np.nan)
    .otherwise(pl.col("factor_flags"))
    .alias("factor_flags"),
    pl.when(pl.col(rain_col).is_nan())
    .then(np.nan)
    .otherwise(pl.col("precip_flags"))
    .alias("precip_flags"),
)

# QC16 - Daily neighbours (wet)
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- Many, although mainly this implementation opens the neighbour wet and dry functions to parameter tweaking

#### Step 1. resample to daily resolution

In [69]:
## resample into daily (also round to 1 decimal place) TODO: check offset='7h' part
target_gauge_daily = (
    target_gauge.group_by_dynamic("time", every="1d", offset="7h", closed="right")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).mean().round(1).alias(rain_col),
        ]
    )
    .filter(pl.col("n_hours") == 24)
    .drop("n_hours")
)  # Ensure at least 24 data points

In [70]:
target_gauge_daily.head()

time,rain_mm
datetime[μs],f64
2006-01-01 07:00:00,0.2
2006-01-02 07:00:00,0.0
2006-01-03 07:00:00,0.0
2006-01-04 07:00:00,0.0
2006-01-05 07:00:00,0.0


#### Step 2. Join each GPCC data & compare to target

In [71]:
existing_gpcc_daily_paths

{'DE_04313': [],
 'DE_04488': [],
 'DE_02718': [],
 'DE_06303': ['../data/GPCC/tw_6303.zip'],
 'DE_00310': ['../data/GPCC/tw_310.zip'],
 'DE_00390': [],
 'DE_03215': ['../data/GPCC/tw_3215.zip'],
 'DE_01300': [],
 'DE_06264': []}

In [72]:
all_data = target_gauge_daily.clone()

for gpcc_id_to_use, gpcc_path in existing_gpcc_daily_paths.items():
    ## 0. Load GPCC data
    gpcc_id_name = gpcc_id_to_use.split("DE_")[1].lstrip("0")
    gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id_name}"
    if len(gpcc_path) == 0:
        continue
    f = zipfile.ZipFile(gpcc_path[0]).open(f"tw_{gpcc_id_name}.dat")
    example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

    ## 1. drop unnecessary columns
    example_gpcc = example_gpcc.drop([str(i) for i in range(4, 16)])

    ## 2. make datetime column (apparently it's 7am-7pm)
    example_gpcc = example_gpcc.with_columns(
        pl.datetime(pl.col("2"), pl.col("1"), pl.col("0"), 7).alias("time")
    ).drop(["0", "1", "2"])

    ## 3. rename and reorder
    example_gpcc = example_gpcc.rename({"3": rain_col})
    example_gpcc = example_gpcc.select(["time", rain_col])  ## Reorder (to look nice)

    ## 4. join to target data
    joined_gauges_gpcc = target_gauge_daily.join(
        example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
    )
    joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()

    ## 5. get normalised diff
    joined_gauges_gpcc_normalised_diff = joined_gauges_gpcc.with_columns(
        # get normalised difference between target and neighbour
        rain_mm_normalised_diff=normalise_data(pl.col(f"{rain_col}"))
        - normalise_data(pl.col(f"{rain_col}_GPCC_{gpcc_id_name}"))
    )

    ## 6. filter values
    joined_gauges_gpcc_normalised_diff_filtered = (
        joined_gauges_gpcc_normalised_diff.filter(
            (pl.col(f"{rain_col}") >= WET_THRESHOLD)
            & (pl.col(f"{rain_col}").is_finite())
            & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}").is_finite())
            & (pl.col(f"{rain_col}_normalised_diff") > 0.0)
        )
    )

    ## 7. Calculate exponential function of normalised diff
    expon_params = scipy.stats.expon.fit(
        joined_gauges_gpcc_normalised_diff_filtered[f"{rain_col}_normalised_diff"]
    )
    # 8. Calculate thresholds at key percentiles of fitted distribution
    q95 = scipy.stats.expon.ppf(0.95, expon_params[0], expon_params[1])
    q99 = scipy.stats.expon.ppf(0.99, expon_params[0], expon_params[1])
    q999 = scipy.stats.expon.ppf(0.999, expon_params[0], expon_params[1])

    ## 9. Assign flags
    joined_gauges_gpcc_normalised_wet_flags = (
        joined_gauges_gpcc_normalised_diff.with_columns(
            pl.when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") <= q95)
            )
            .then(0)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q95)
                & (pl.col(f"{rain_col}_normalised_diff") <= q99),
            )
            .then(1)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q99)
                & (pl.col(f"{rain_col}_normalised_diff") <= q999),
            )
            .then(2)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q95)
            )
            .then(3)
            .otherwise(0)
            .alias(f"wet_flags_{gpcc_id_name}")
        )
    )

    ## 10. Join to all data
    all_data = all_data.join(
        joined_gauges_gpcc_normalised_wet_flags[["time", f"wet_flags_{gpcc_id_name}"]],
        on="time",
        how="left",
    )
    print(
        joined_gauges_gpcc_normalised_wet_flags[
            f"wet_flags_{gpcc_id_name}"
        ].value_counts()
    )

shape: (4, 2)
┌────────────────┬───────┐
│ wet_flags_6303 ┆ count │
│ ---            ┆ ---   │
│ i32            ┆ u32   │
╞════════════════╪═══════╡
│ 3              ┆ 1     │
│ 1              ┆ 1     │
│ 2              ┆ 1     │
│ 0              ┆ 1756  │
└────────────────┴───────┘
shape: (4, 2)
┌───────────────┬───────┐
│ wet_flags_310 ┆ count │
│ ---           ┆ ---   │
│ i32           ┆ u32   │
╞═══════════════╪═══════╡
│ 3             ┆ 1     │
│ 2             ┆ 1     │
│ 0             ┆ 1756  │
│ 1             ┆ 1     │
└───────────────┴───────┘
shape: (4, 2)
┌────────────────┬───────┐
│ wet_flags_3215 ┆ count │
│ ---            ┆ ---   │
│ i32            ┆ u32   │
╞════════════════╪═══════╡
│ 0              ┆ 1250  │
│ 1              ┆ 4     │
│ 3              ┆ 1     │
│ 2              ┆ 1     │
└────────────────┴───────┘


#### Step 3. Calculate num neighbours online

In [73]:
## global determining the minimum number of non-null neighbours to count flags
MIN_NUM_NEIGHBOURS = 2  # original method is 3

In [74]:
num_neighbours = len(all_data.columns[2:])
num_neighbours

3

In [75]:
all_data_num_neighbours = all_data.with_columns(
    (
        num_neighbours
        - pl.sum_horizontal(pl.all().exclude("time").is_null().cast(pl.Int16))
    ).alias("num_neighbours_online")
)
all_data_num_neighbours

time,rain_mm,wet_flags_6303,wet_flags_310,wet_flags_3215,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i16
2006-01-01 07:00:00,0.2,0,0,null,2
2006-01-02 07:00:00,0.0,0,0,null,2
2006-01-03 07:00:00,0.0,0,0,null,2
2006-01-04 07:00:00,0.0,0,0,null,2
2006-01-05 07:00:00,0.0,0,0,null,2
…,…,…,…,…,…
2010-12-26 07:00:00,0.2,0,0,0,3
2010-12-27 07:00:00,0.0,0,0,0,3
2010-12-28 07:00:00,0.0,0,0,0,3


In [76]:
all_data_num_neighbours["num_neighbours_online"].plot.line()

alt.Chart(...)

#### Step 4. Majority voting flags for the target

In [77]:
all_data_wet_flags = all_data_num_neighbours.with_columns(
    pl.when(pl.col("num_neighbours_online") < MIN_NUM_NEIGHBOURS)
    .then(np.nan)
    .otherwise(
        pl.min_horizontal(pl.all().exclude("time", rain_col, "num_neighbours_online"))
    )
    .alias("wet_flags")
)

In [78]:
all_data_wet_flags["wet_flags"].value_counts()

wet_flags,count
f64,u32
1.0,1
2.0,1
3.0,1
NaN,66
0.0,1756


##### Extra note about method:
We are checking for minimum 2 neighbours, but original method needs 3

In [79]:
all_data_num_neighbours.filter(pl.col("wet_flags_310") == 3)

time,rain_mm,wet_flags_6303,wet_flags_310,wet_flags_3215,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i16
2006-12-24 07:00:00,300.0,3,3,null,2


# QC17 - Hourly neighbours (wet) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- Need to consider how the 7am-7pm can fit into RainfallQC

#### Step 0. resample to daily resolution

In [80]:
## resample into daily (also round to 1 decimal place) TODO: check offset='7h' part
target_gauge_daily = (
    target_gauge.group_by_dynamic("time", every="1d", offset="7h", closed="right")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).mean().round(1).alias(rain_col),
        ]
    )
    .filter(pl.col("n_hours") == 24)
    .drop("n_hours")
)  # Ensure at least 24 data points

In [81]:
target_gauge_daily.head()

time,rain_mm
datetime[μs],f64
2006-01-01 07:00:00,0.2
2006-01-02 07:00:00,0.0
2006-01-03 07:00:00,0.0
2006-01-04 07:00:00,0.0
2006-01-05 07:00:00,0.0


#### Step 1. Loop through each neighbour and compare to target

In [82]:
all_data = (
    target_gauge_daily.clone()
)  ## currently daily instead of hourly because this works best with later flood-fill

for n_id in all_neighbour_ids_paths.keys():
    ## 1. Get neighbouring GDSR data nd resample into daily
    print(n_id)
    neighbouring_gdsr_gauge = get_neighbouring_gdsr_data(
        neighbour_gdsr_id=n_id, time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
    )
    ## make no data vals nans
    station_metadata = read_metadata(data_path=all_neighbour_ids_paths[n_id])
    neighbouring_gdsr_gauge = replace_no_data_with_nan(
        neighbouring_gdsr_gauge, no_data_value=int(station_metadata["no_data_value"])
    )

    joined_gauges_gdsr = target_gauge.join(
        neighbouring_gdsr_gauge, on="time", suffix=f"_{n_id}"
    )
    joined_gauges_gdsr = joined_gauges_gdsr.drop_nans()

    ## resample into daily (also round to 1 decimal place) TODO: think about offset part
    joined_gauges_gdsr = (
        joined_gauges_gdsr.group_by_dynamic("time", every="1d", offset="7h")
        .agg(
            [
                pl.len().alias("n_hours"),
                pl.col(rain_col).mean().round(1).alias(rain_col),
                pl.col(f"{rain_col}_{n_id}")
                .mean()
                .round(1)
                .alias(f"{rain_col}_GDSR_{n_id}"),
            ]
        )
        .filter(pl.col("n_hours") == 24)
        .drop("n_hours")
    )  # Ensure at least 24 data points

    ## 2. get normalised diff
    joined_gauges_gdsr_normalised_diff = joined_gauges_gdsr.with_columns(
        # get normalised difference between target and neighbour
        rain_mm_normalised_diff=normalise_data(pl.col(f"{rain_col}"))
        - normalise_data(pl.col(f"{rain_col}_GDSR_{n_id}"))
    )

    ## 3. filter values
    joined_gauges_gdsr_normalised_diff_filtered = (
        joined_gauges_gdsr_normalised_diff.filter(
            (pl.col(f"{rain_col}") >= WET_THRESHOLD)
            & (pl.col(f"{rain_col}").is_finite())
            & (pl.col(f"{rain_col}_GDSR_{n_id}").is_finite())
            & (pl.col(f"{rain_col}_normalised_diff") > 0.0)
        )
    )

    ## 4. Calcualte exponetial function of normalised diff
    expon_params = scipy.stats.expon.fit(
        joined_gauges_gdsr_normalised_diff_filtered[f"{rain_col}_normalised_diff"]
    )
    # 5. Calculate thresholds at key percentiles of fitted distribution
    q95 = scipy.stats.expon.ppf(0.95, expon_params[0], expon_params[1])
    q99 = scipy.stats.expon.ppf(0.99, expon_params[0], expon_params[1])
    q999 = scipy.stats.expon.ppf(0.999, expon_params[0], expon_params[1])

    ## 6. Assign flags
    joined_gauges_gdsr_normalised_wet_flags = (
        joined_gauges_gdsr_normalised_diff.with_columns(
            pl.when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") <= q95)
            )
            .then(0)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q95)
                & (pl.col(f"{rain_col}_normalised_diff") <= q99),
            )
            .then(1)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q99)
                & (pl.col(f"{rain_col}_normalised_diff") <= q999),
            )
            .then(2)
            .when(
                (pl.col(rain_col) >= 1.0)
                & (pl.col(f"{rain_col}_normalised_diff") > q95)
            )
            .then(3)
            .otherwise(0)
            .alias(f"wet_flags_{n_id}")
        )
    )
    all_data = all_data.join(
        joined_gauges_gdsr_normalised_wet_flags[["time", f"wet_flags_{n_id}"]],
        on="time",
        how="left",
    )

    print(
        "before turning back into hourly", all_data[f"wet_flags_{n_id}"].value_counts()
    )

DE_04313
before turning back into hourly shape: (3, 2)
┌────────────────────┬───────┐
│ wet_flags_DE_04313 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ null               ┆ 71    │
│ 3                  ┆ 3     │
│ 0                  ┆ 1751  │
└────────────────────┴───────┘
DE_04488
before turning back into hourly shape: (4, 2)
┌────────────────────┬───────┐
│ wet_flags_DE_04488 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 0                  ┆ 1548  │
│ 3                  ┆ 2     │
│ 2                  ┆ 1     │
│ null               ┆ 274   │
└────────────────────┴───────┘
DE_02718
before turning back into hourly shape: (3, 2)
┌────────────────────┬───────┐
│ wet_flags_DE_02718 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 3                  ┆ 3     │
│ null               ┆ 68    │
│ 0                  ┆ 1754  

#### Step 2. Calculate num neighbours online

In [83]:
## global determining the minimum number of non-null neighbours to count flags
MIN_NUM_NEIGHBOURS = 2  # original method is 3

In [84]:
all_data.head()

time,rain_mm,wet_flags_DE_04313,wet_flags_DE_04488,wet_flags_DE_02718,wet_flags_DE_06303,wet_flags_DE_00310,wet_flags_DE_00390,wet_flags_DE_03215,wet_flags_DE_01300,wet_flags_DE_06264
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32
2006-01-01 07:00:00,0.2,0,null,0,0,0,0,null,0,0
2006-01-02 07:00:00,0.0,0,null,0,0,0,0,null,0,0
2006-01-03 07:00:00,0.0,0,null,0,0,0,0,null,0,0
2006-01-04 07:00:00,0.0,0,null,0,0,0,0,null,0,0
2006-01-05 07:00:00,0.0,0,null,0,0,0,0,null,0,0


In [85]:
num_neighbours = len(all_data.columns[2:])
num_neighbours

9

In [86]:
all_data_num_neighbours = all_data.with_columns(
    (
        num_neighbours
        - pl.sum_horizontal(pl.all().exclude("time").is_null().cast(pl.Int16))
    ).alias("num_neighbours_online")
)
all_data_num_neighbours

time,rain_mm,wet_flags_DE_04313,wet_flags_DE_04488,wet_flags_DE_02718,wet_flags_DE_06303,wet_flags_DE_00310,wet_flags_DE_00390,wet_flags_DE_03215,wet_flags_DE_01300,wet_flags_DE_06264,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i16
2006-01-01 07:00:00,0.2,0,null,0,0,0,0,null,0,0,7
2006-01-02 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-03 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-04 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-05 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
…,…,…,…,…,…,…,…,…,…,…,…
2010-12-26 07:00:00,0.2,0,0,0,0,0,0,0,0,0,9
2010-12-27 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9
2010-12-28 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9


In [87]:
all_data_num_neighbours["num_neighbours_online"].value_counts()

num_neighbours_online,count
i16,u32
6,9
4,1
0,65
8,335
5,1
9,1210
7,203
3,1


In [88]:
all_data.filter(pl.col("wet_flags_DE_00310") == 2)[0]

time,rain_mm,wet_flags_DE_04313,wet_flags_DE_04488,wet_flags_DE_02718,wet_flags_DE_06303,wet_flags_DE_00310,wet_flags_DE_00390,wet_flags_DE_03215,wet_flags_DE_01300,wet_flags_DE_06264
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32
2006-12-23 07:00:00,131.3,3,2,3,2,2,3,null,2,3


#### Step 3. Majority voting flags for the target

In [89]:
all_data_wet_flags = all_data_num_neighbours.with_columns(
    pl.when(pl.col("num_neighbours_online") < MIN_NUM_NEIGHBOURS)
    .then(np.nan)
    .otherwise(
        pl.min_horizontal(pl.all().exclude("time", rain_col, "num_neighbours_online"))
    )
    .alias("wet_flags")
)

In [90]:
all_data_wet_flags["wet_flags"].value_counts()

wet_flags,count
f64,u32
3.0,2
2.0,1
0.0,1757
NaN,65


## Step 4. Join back to hourly data

In [91]:
target_gauge_w_wet_flags = target_gauge.join(
    all_data_wet_flags[["time", "wet_flags"]], on="time", how="left"
)

## Step 5. Forward flood-fill

In [92]:
## Forward flood-fill data to convert the flags back to hourly
target_gauge_w_wet_flags = target_gauge_w_wet_flags.with_columns(
    pl.col("wet_flags").forward_fill(limit=23)
)

In [93]:
target_gauge_w_wet_flags["wet_flags"].value_counts()

wet_flags,count
f64,u32
NaN,1560
3.0,48
0.0,42168
null,24
2.0,24


# QC18 - Daily neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [94]:
DRY_PERIOD_DAYS = 15

In [95]:
fraction_drydays = {}
for d in range(1, 3 + 1):
    fraction_drydays[str(d)] = np.trunc(
        (1.0 - (float(d) / DRY_PERIOD_DAYS)) * 10**2
    ) / (10**2)

#### Step 1. Loop through each neighbour and compare to target

In [96]:
all_data = target_gauge_daily.clone()

for gpcc_id_to_use, gpcc_path in existing_gpcc_daily_paths.items():
    ## 0. Load GPCC data
    gpcc_id_name = gpcc_id_to_use.split("DE_")[1].lstrip("0")
    gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id_name}"
    if len(gpcc_path) == 0:
        continue
    f = zipfile.ZipFile(gpcc_path[0]).open(f"tw_{gpcc_id_name}.dat")
    example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

    ## 1. drop unnecessary columns
    example_gpcc = example_gpcc.drop([str(i) for i in range(4, 16)])

    ## 2. make datetime column (apparently it's 7am-7pm)
    example_gpcc = example_gpcc.with_columns(
        pl.datetime(pl.col("2"), pl.col("1"), pl.col("0"), 7).alias("time")
    ).drop(["0", "1", "2"])

    ## 3. rename and reorder
    example_gpcc = example_gpcc.rename({"3": rain_col})
    example_gpcc = example_gpcc.select(["time", rain_col])  ## Reorder (to look nice)

    ## 4. join to target data
    joined_gauges_gpcc = target_gauge_daily.join(
        example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
    )
    joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()

    ## 5. convert to dry spell
    joined_gauges_gpcc_dry_spell = joined_gauges_gpcc.with_columns(
        pl.col(rain_col)
        .map_batches(
            lambda row: convert_to_dry_spell(row, DRY_PERIOD_DAYS, col_name=rain_col)
        )
        .alias("dry_spell_fraction"),
        pl.col(gpcc_col_name)
        .map_batches(
            lambda row: convert_to_dry_spell(
                row, DRY_PERIOD_DAYS, col_name=gpcc_col_name
            )
        )
        .alias(f"dry_spell_fraction_{gpcc_id}"),
    )

    ## 7. Assign dry flags
    joined_gauges_gpcc_dry_flags = joined_gauges_gpcc_dry_spell.with_columns(
        pl.when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") == 1.0)
        )
        .then(0)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["1"]),
        )
        .then(1)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["1"])
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["2"]),
        )
        .then(2)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["2"])
        )
        .then(3)
        .otherwise(0)
        .alias(f"dry_flags_{gpcc_id_to_use}")
    )

    print(
        gpcc_id_to_use,
        joined_gauges_gpcc_dry_flags[f"dry_flags_{gpcc_id_to_use}"].value_counts(),
    )

    ## 8. Join to all data
    all_data = all_data.join(
        joined_gauges_gpcc_dry_flags[["time", f"dry_flags_{gpcc_id_to_use}"]],
        on="time",
        how="left",
    )

DE_06303 shape: (3, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_06303 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 2                  ┆ 1     │
│ 0                  ┆ 1628  │
│ 3                  ┆ 130   │
└────────────────────┴───────┘
DE_00310 shape: (4, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_00310 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 1                  ┆ 1     │
│ 0                  ┆ 1628  │
│ 3                  ┆ 126   │
│ 2                  ┆ 4     │
└────────────────────┴───────┘
DE_03215 shape: (4, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_03215 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 3                  ┆ 126   │
│ 1                  ┆ 1     │
│ 2                  ┆ 4     │
│ 0                  ┆ 1125  │
└────────────────────┴───────┘


#### Step 2. Calculate num neighbours online

In [97]:
## global determining the minimum number of non-null neighbours to count flags
MIN_NUM_NEIGHBOURS = 2  # original method is 3

In [98]:
all_data.head()

time,rain_mm,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_03215
datetime[μs],f64,i32,i32,i32
2006-01-01 07:00:00,0.2,0,0,null
2006-01-02 07:00:00,0.0,0,0,null
2006-01-03 07:00:00,0.0,0,0,null
2006-01-04 07:00:00,0.0,0,0,null
2006-01-05 07:00:00,0.0,0,0,null


In [99]:
num_neighbours = len(all_data.columns[2:])
num_neighbours

3

In [100]:
all_data_num_neighbours = all_data.with_columns(
    (
        num_neighbours
        - pl.sum_horizontal(pl.all().exclude("time").is_null().cast(pl.Int16))
    ).alias("num_neighbours_online")
)
all_data_num_neighbours

time,rain_mm,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_03215,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i16
2006-01-01 07:00:00,0.2,0,0,null,2
2006-01-02 07:00:00,0.0,0,0,null,2
2006-01-03 07:00:00,0.0,0,0,null,2
2006-01-04 07:00:00,0.0,0,0,null,2
2006-01-05 07:00:00,0.0,0,0,null,2
…,…,…,…,…,…
2010-12-26 07:00:00,0.2,0,0,0,3
2010-12-27 07:00:00,0.0,0,0,0,3
2010-12-28 07:00:00,0.0,0,0,0,3


In [101]:
all_data_num_neighbours["num_neighbours_online"].value_counts()

num_neighbours_online,count
i16,u32
3,1256
2,503
0,66


In [102]:
all_data.filter(pl.col("dry_flags_DE_03215") == 1)

time,rain_mm,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_03215
datetime[μs],f64,i32,i32,i32
2010-07-04 07:00:00,0.0,3,1,1


#### Step 3. Majority voting flags for the target

In [103]:
all_data_dry_flags = all_data_num_neighbours.with_columns(
    pl.when(pl.col("num_neighbours_online") < MIN_NUM_NEIGHBOURS)
    .then(np.nan)
    .otherwise(
        pl.min_horizontal(pl.all().exclude("time", rain_col, "num_neighbours_online"))
    )
    .alias("dry_flags")
)

In [104]:
all_data_dry_flags["dry_flags"].value_counts()

dry_flags,count
f64,u32
1.0,1
3.0,125
NaN,66
0.0,1628
2.0,5


In [105]:
all_data_dry_flags.filter((pl.col("dry_flags") > 0) & (pl.col("dry_flags").is_finite()))

time,rain_mm,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_03215,num_neighbours_online,dry_flags
datetime[μs],f64,i32,i32,i32,i16,f64
2007-10-16 07:00:00,0.0,3,2,2,3,2.0
2007-12-26 07:00:00,0.0,3,2,2,3,2.0
2007-12-27 07:00:00,0.0,3,3,3,3,3.0
2007-12-28 07:00:00,0.0,2,2,3,3,2.0
2008-05-13 07:00:00,0.0,3,2,2,3,2.0
…,…,…,…,…,…,…
2010-10-26 07:00:00,0.0,3,3,3,3,3.0
2010-10-27 07:00:00,0.0,3,3,3,3,3.0
2010-10-28 07:00:00,0.0,3,3,3,3,3.0


### Step 4. Back propogate flags so period previous to flag is also given the same flag
Number of days of period set by DRY_PERIOD_DAYS

In [106]:
def back_propagate_flags(df, flag_column, num_days):
    """
    Back fill-in flags a number of days. This will prioritise higher flag values

    Parameters
    ----------
    df : polars.DataFrame
        Data with `flag_column` series
    flag_column : str
        column with flags
    num_days: int
        Number of days to back-propagate
    Returns
    -------
    df

    """
    df = df.clone()
    # TODO: automatically extract flag values
    flag_values = (
        flag for flag in df[flag_column].unique() if (np.isfinite(flag) & (flag > 0))
    )
    for flag, df_filtered in [
        (flag, df.filter(pl.col(flag_column) == flag)) for flag in flag_values
    ]:
        for time_to_use in df_filtered["time"]:
            assert isinstance(
                time_to_use, datetime.datetime
            ), "time_to_use must be datetime.datetime"
            start_time = time_to_use - datetime.timedelta(days=num_days)
            df = df.with_columns(
                pl.when(
                    (pl.col("time") >= start_time) & (pl.col("time") <= time_to_use)
                )
                .then(flag)
                .otherwise(pl.col(flag_column))
                .alias(flag_column)
            )

    return df

In [107]:
all_data_dry_flags_back_prop = back_propagate_flags(
    all_data_dry_flags, flag_column="dry_flags", num_days=(DRY_PERIOD_DAYS - 1)
)

In [108]:
## before:
all_data_dry_flags["dry_flags"].value_counts()

dry_flags,count
f64,u32
2.0,5
0.0,1628
1.0,1
3.0,125
NaN,66


In [109]:
## after
all_data_dry_flags_back_prop["dry_flags"].value_counts()

dry_flags,count
f64,u32
1.0,15
0.0,1516
2.0,33
NaN,66
3.0,195


In [110]:
all_data_dry_flags_back_prop[845:950]

time,rain_mm,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_03215,num_neighbours_online,dry_flags
datetime[μs],f64,i32,i32,i32,i16,f64
2008-04-25 07:00:00,0.0,0,0,0,3,0.0
2008-04-26 07:00:00,0.0,0,0,0,3,0.0
2008-04-27 07:00:00,0.0,0,0,0,3,0.0
2008-04-28 07:00:00,0.6,0,0,0,3,0.0
2008-04-29 07:00:00,0.0,0,0,0,3,2.0
…,…,…,…,…,…,…
2008-08-03 07:00:00,0.7,0,0,0,3,0.0
2008-08-04 07:00:00,0.1,0,0,0,3,0.0
2008-08-05 07:00:00,0.0,0,0,0,3,0.0


# QC19 - Hourly neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- NOTE: the order in which the data is back-propogated and forward flood-fill will matter
- TODO: sort out whether you want to do back-propogation or forward flood fill first (original metho does back prop then flood fill)

#### Step 0. resample to daily resolution

In [111]:
## resample into daily (also round to 1 decimal place) TODO: check offset='7h' part
target_gauge_daily = (
    target_gauge.group_by_dynamic("time", every="1d", offset="7h", closed="right")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).mean().round(1).alias(rain_col),
        ]
    )
    .filter(pl.col("n_hours") == 24)
    .drop("n_hours")
)  # Ensure at least 24 data points

In [112]:
target_gauge_daily.head()

time,rain_mm
datetime[μs],f64
2006-01-01 07:00:00,0.2
2006-01-02 07:00:00,0.0
2006-01-03 07:00:00,0.0
2006-01-04 07:00:00,0.0
2006-01-05 07:00:00,0.0


## Step 1. Loop through each neighbour

In [113]:
all_data = (
    target_gauge_daily.clone()
)  ## currently daily instead of hourly because this works best with later flood-fill

for n_id in all_neighbour_ids_paths.keys():
    ## 0. Get neighbouring GDSR data
    print(n_id)
    gdsr_col_name = f"{rain_col}_GDSR_{n_id}"
    neighbouring_gdsr_gauge = get_neighbouring_gdsr_data(
        neighbour_gdsr_id=n_id, time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
    )
    ## 1. make no data vals nans
    station_metadata = read_metadata(data_path=all_neighbour_ids_paths[n_id])
    neighbouring_gdsr_gauge = replace_no_data_with_nan(
        neighbouring_gdsr_gauge, no_data_value=int(station_metadata["no_data_value"])
    )

    joined_gauges_gdsr = target_gauge.join(
        neighbouring_gdsr_gauge, on="time", suffix=f"_{n_id}"
    )
    joined_gauges_gdsr = joined_gauges_gdsr.drop_nans()

    ## 2. resample into daily (also round to 1 decimal place) TODO: think about offset part
    joined_gauges_gdsr = (
        joined_gauges_gdsr.group_by_dynamic("time", every="1d", offset="7h")
        .agg(
            [
                pl.len().alias("n_hours"),
                pl.col(rain_col).mean().round(1).alias(rain_col),
                pl.col(f"{rain_col}_{n_id}").mean().round(1).alias(gdsr_col_name),
            ]
        )
        .filter(pl.col("n_hours") == 24)
        .drop("n_hours")
    )  # Ensure at least 24 data points

    ## 3. convert to dry spell
    joined_gauges_gdsr_dry_spell = joined_gauges_gdsr.with_columns(
        pl.col(rain_col)
        .map_batches(
            lambda row: convert_to_dry_spell(row, DRY_PERIOD_DAYS, col_name=rain_col)
        )
        .alias("dry_spell_fraction"),
        pl.col(gdsr_col_name)
        .map_batches(
            lambda row: convert_to_dry_spell(
                row, DRY_PERIOD_DAYS, col_name=gdsr_col_name
            )
        )
        .alias(f"dry_spell_fraction_{n_id}"),
    )

    ## 4. Assign dry flags
    joined_gauges_gdsr_dry_flags = joined_gauges_gdsr_dry_spell.with_columns(
        pl.when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{n_id}") == 1.0)
        )
        .then(0)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{n_id}") < 1.0)
            & (pl.col(f"dry_spell_fraction_{n_id}") >= fraction_drydays["1"]),
        )
        .then(1)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{n_id}") < fraction_drydays["1"])
            & (pl.col(f"dry_spell_fraction_{n_id}") >= fraction_drydays["2"]),
        )
        .then(2)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{n_id}") < fraction_drydays["2"])
        )
        .then(3)
        .otherwise(0)
        .alias(f"dry_flags_{n_id}")
    )

    ## 5. Join to all data
    all_data = all_data.join(
        joined_gauges_gdsr_dry_flags[["time", f"dry_flags_{n_id}"]],
        on="time",
        how="left",
    )

    print(
        "Hourly grouped into daily:",
        gpcc_id_to_use,
        joined_gauges_gdsr_dry_flags[f"dry_flags_{n_id}"].value_counts(),
    )

DE_04313
Hourly grouped into daily: DE_06264 shape: (4, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_04313 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 0                  ┆ 1626  │
│ 2                  ┆ 23    │
│ 3                  ┆ 103   │
│ 1                  ┆ 2     │
└────────────────────┴───────┘
DE_04488
Hourly grouped into daily: DE_06264 shape: (4, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_04488 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 1                  ┆ 4     │
│ 3                  ┆ 103   │
│ 0                  ┆ 1425  │
│ 2                  ┆ 19    │
└────────────────────┴───────┘
DE_02718
Hourly grouped into daily: DE_06264 shape: (4, 2)
┌────────────────────┬───────┐
│ dry_flags_DE_02718 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 3                  ┆ 110   │
│ 2              

## Step 2. Calculate num neighbours online

In [114]:
## global determining the minimum number of non-null neighbours to count flags
MIN_NUM_NEIGHBOURS = 2  # original method is 3

In [115]:
num_neighbours = len(all_data.columns[2:])
num_neighbours

9

In [116]:
all_data_num_neighbours = all_data.with_columns(
    (
        num_neighbours
        - pl.sum_horizontal(pl.all().exclude("time").is_null().cast(pl.Int16))
    ).alias("num_neighbours_online")
)
all_data_num_neighbours

time,rain_mm,dry_flags_DE_04313,dry_flags_DE_04488,dry_flags_DE_02718,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_00390,dry_flags_DE_03215,dry_flags_DE_01300,dry_flags_DE_06264,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i16
2006-01-01 07:00:00,0.2,0,null,0,0,0,0,null,0,0,7
2006-01-02 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-03 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-04 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
2006-01-05 07:00:00,0.0,0,null,0,0,0,0,null,0,0,7
…,…,…,…,…,…,…,…,…,…,…,…
2010-12-26 07:00:00,0.2,0,0,0,0,0,0,0,0,0,9
2010-12-27 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9
2010-12-28 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9


In [117]:
all_data_num_neighbours["num_neighbours_online"].value_counts()

num_neighbours_online,count
i16,u32
0,65
3,1
9,1210
8,335
5,1
6,9
4,1
7,203


In [118]:
all_data.filter(pl.col("dry_flags_DE_03215") == 1)

time,rain_mm,dry_flags_DE_04313,dry_flags_DE_04488,dry_flags_DE_02718,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_00390,dry_flags_DE_03215,dry_flags_DE_01300,dry_flags_DE_06264
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32
2007-10-16 07:00:00,0.0,1,0,0,0,0,null,1,null,0
2008-09-28 07:00:00,0.0,2,0,3,2,1,1,1,1,2
2008-09-29 07:00:00,0.0,2,2,3,3,1,1,1,1,2
2009-07-02 07:00:00,0.0,2,2,3,2,3,3,1,3,3
2009-07-03 07:00:00,0.0,2,2,3,2,3,3,1,3,3
2009-07-04 07:00:00,0.0,2,2,3,2,3,3,1,3,3


## Step 3. Majority voting flags for the target

In [119]:
all_data_dry_flags = all_data_num_neighbours.with_columns(
    pl.when(pl.col("num_neighbours_online") < MIN_NUM_NEIGHBOURS)
    .then(np.nan)
    .otherwise(
        pl.min_horizontal(pl.all().exclude("time", rain_col, "num_neighbours_online"))
    )
    .alias("dry_flags")
)

In [120]:
all_data_dry_flags["dry_flags"].value_counts()

dry_flags,count
f64,u32
2.0,25
3.0,92
NaN,65
0.0,1637
1.0,6


In [121]:
all_data_dry_flags.filter((pl.col("dry_flags") > 0) & (pl.col("dry_flags").is_finite()))

time,rain_mm,dry_flags_DE_04313,dry_flags_DE_04488,dry_flags_DE_02718,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_00390,dry_flags_DE_03215,dry_flags_DE_01300,dry_flags_DE_06264,num_neighbours_online,dry_flags
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i16,f64
2008-09-29 07:00:00,0.0,2,2,3,3,1,1,1,1,2,9,1.0
2008-09-30 07:00:00,0.0,3,3,3,3,2,2,2,2,3,9,2.0
2008-10-01 07:00:00,0.0,3,3,3,null,3,3,3,3,3,8,3.0
2008-10-02 07:00:00,0.0,3,3,3,3,3,3,3,3,null,8,3.0
2008-10-03 07:00:00,0.0,3,3,3,3,3,3,3,3,3,9,3.0
…,…,…,…,…,…,…,…,…,…,…,…,…
2010-10-26 07:00:00,0.0,3,3,3,3,3,3,3,3,3,9,3.0
2010-10-27 07:00:00,0.0,3,3,3,3,3,3,3,3,3,9,3.0
2010-10-28 07:00:00,0.0,3,3,3,3,3,3,3,3,3,9,3.0


## Step 4. Back propogate flags so period previous to flag is also given the same flag
Number of days of period set by DRY_PERIOD_DAYS

In [122]:
def back_propagate_flags(df, flag_column, num_days):
    """
    Back fill-in flags a number of days. This will prioritise higher flag values

    Parameters
    ----------
    df : polars.DataFrame
        Data with `flag_column` series
    flag_column : str
        column with flags
    num_days: int
        Number of days to back-propagate
    Returns
    -------
    df

    """
    df = df.clone()
    # TODO: automatically extract flag values
    flag_values = (
        flag for flag in df[flag_column].unique() if (np.isfinite(flag) & (flag > 0))
    )
    for flag, df_filtered in [
        (flag, df.filter(pl.col(flag_column) == flag)) for flag in flag_values
    ]:
        for time_to_use in df_filtered["time"]:
            assert isinstance(
                time_to_use, datetime.datetime
            ), "time_to_use must be datetime.datetime"
            start_time = time_to_use - datetime.timedelta(days=num_days)
            df = df.with_columns(
                pl.when(
                    (pl.col("time") >= start_time) & (pl.col("time") <= time_to_use)
                )
                .then(flag)
                .otherwise(pl.col(flag_column))
                .alias(flag_column)
            )

    return df

In [123]:
all_data_dry_flags_back_prop = back_propagate_flags(
    all_data_dry_flags, flag_column="dry_flags", num_days=(DRY_PERIOD_DAYS - 1)
)

In [124]:
## whilst still daily data:
all_data_dry_flags["dry_flags"].value_counts()

dry_flags,count
f64,u32
2.0,25
0.0,1637
3.0,92
1.0,6
NaN,65


In [125]:
## after
all_data_dry_flags_back_prop["dry_flags"].value_counts()

dry_flags,count
f64,u32
1.0,1
2.0,8
NaN,65
3.0,170
0.0,1581


In [126]:
all_data_dry_flags_back_prop[845:950]

time,rain_mm,dry_flags_DE_04313,dry_flags_DE_04488,dry_flags_DE_02718,dry_flags_DE_06303,dry_flags_DE_00310,dry_flags_DE_00390,dry_flags_DE_03215,dry_flags_DE_01300,dry_flags_DE_06264,num_neighbours_online,dry_flags
datetime[μs],f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i16,f64
2008-04-25 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9,0.0
2008-04-26 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9,0.0
2008-04-27 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9,0.0
2008-04-28 07:00:00,0.6,0,0,0,0,0,0,0,0,0,9,0.0
2008-04-29 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…
2008-08-03 07:00:00,0.7,0,0,0,0,0,0,0,0,0,9,0.0
2008-08-04 07:00:00,0.1,0,0,0,0,0,0,0,0,0,9,0.0
2008-08-05 07:00:00,0.0,0,0,0,0,0,0,0,0,0,9,0.0


## Step 5. Join back to hourly data

In [127]:
target_gauge_w_dry_flags = target_gauge.join(
    all_data_dry_flags_back_prop[["time", "dry_flags"]], on="time", how="left"
)

## Step 6. Forward flood-fill

In [128]:
## Forward flood-fill data to convert the flags back to hourly
target_gauge_w_dry_flags = target_gauge_w_dry_flags.with_columns(
    pl.col("dry_flags").forward_fill(limit=23)
)

In [129]:
target_gauge_w_dry_flags["dry_flags"].value_counts()

dry_flags,count
f64,u32
3.0,4080
null,24
2.0,192
1.0,24
0.0,37944
NaN,1560


In [130]:
target_gauge_w_dry_flags.filter(
    (pl.col("dry_flags") > 0) & (pl.col("dry_flags").is_finite())
)

time,rain_mm,dry_flags
datetime[μs],f64,f64
2008-09-15 07:00:00,0.0,1.0
2008-09-15 08:00:00,0.0,1.0
2008-09-15 09:00:00,0.0,1.0
2008-09-15 10:00:00,0.0,1.0
2008-09-15 11:00:00,0.0,1.0
…,…,…
2010-10-31 02:00:00,0.0,3.0
2010-10-31 03:00:00,0.0,3.0
2010-10-31 04:00:00,0.0,3.0


# QC20 - Monthly neighbours
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- The intenseQC code is wrong for the +/-4, 5 flags, so I am going off the description in the paper

## Step 0. Resample into monthly
Also includes step of extracting year and month and computing expected days in month

In [131]:
PERC_MONTH_NEEDED = 95  ## 95% stated in paper

In [132]:
# NOTE: Annoyingly, there is no days_in_month functionality for polars yet
def make_monthyear_col(data):
    # Make year and month columns
    return data.with_columns(
        [
            pl.col("time").dt.year().alias("year"),
            pl.col("time").dt.month().alias("month"),
        ]
    )


def get_expected_days_in_month(data):
    """

    Parameters
    ----------
    data : pl.DataFrame
        Data with 'year' and 'month' columns

    Returns
    -------

    """
    # Use map_elements + calendar.monthrange to compute days in each month
    return data.with_columns(
        [
            pl.struct(["year", "month"])
            .map_elements(
                lambda x: calendar.monthrange(x["year"], x["month"])[1],
                return_dtype=pl.Int64,
            )
            .alias("expected_days_in_month")
        ]
    )

In [ ]:
target_gauge_monthyear = make_monthyear_col(target_gauge)
target_gauge_monthyear = get_expected_days_in_month(target_gauge_monthyear)

In [133]:
target_gauge_monthly = (
    target_gauge_monthyear.group_by_dynamic("time", every="1mo", closed="right")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).sum().alias(rain_col),
            pl.col("expected_days_in_month").first(),
        ]
    )
    .filter(
        pl.col("n_hours")
        >= (24 * pl.col("expected_days_in_month") * PERC_MONTH_NEEDED / 100)
    )  # TODO: Ensure at least 95% values for month are available
    .drop("n_hours", "expected_days_in_month")
)

In [134]:
target_gauge_monthly

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,NaN
2006-02-01 00:00:00,NaN
2006-03-01 00:00:00,302.9
2006-04-01 00:00:00,224.2
2006-05-01 00:00:00,402.2
…,…
2010-08-01 00:00:00,417.1
2010-09-01 00:00:00,30.0
2010-10-01 00:00:00,40.0


## Step 1. Loop through monthly GPCC neighbours and compare to target

TODO: need to find a way to programatically parse TW (daily) and MW (monthly) data

In [135]:
def calculate_perc_diff(target, other):
    """
    Parameters
    ----------
    target: pl.Series
        Target data to compare other too
    other: pl.Series
        Other data
    Returns
    -------
    perc_diff: pl.Series
    """
    return (target - other) * 100 / other

In [136]:
all_data = target_gauge_monthly.clone()

for gpcc_id_to_use, gpcc_path in existing_gpcc_monthly_paths.items():
    ## Step 0. Load GPCC monthly data
    gpcc_id_name = gpcc_id_to_use.split("DE_")[1].lstrip("0")
    if len(gpcc_path) == 0:
        continue
    f = zipfile.ZipFile(gpcc_path[0]).open(f"mw_{gpcc_id_name}.dat")
    example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

    ## Step 1. drop unnecessary columns
    example_gpcc = example_gpcc.drop([str(i) for i in range(3, 15)])

    ## Step 2. make datetime column (apparently it's 7am-7pm)
    example_gpcc = example_gpcc.with_columns(
        pl.datetime(year=pl.col("1"), month=pl.col("0"), day=1).alias("time")
    ).drop(["0", "1"])

    ## Step 3. rename and reorder
    example_gpcc = example_gpcc.rename({"2": rain_col})
    example_gpcc = example_gpcc.select(["time", rain_col])  ## Reorder (to look nice)

    ## Step 4. join to target data
    joined_gauges_gpcc = target_gauge_monthly.join(
        example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
    )
    joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()

    ## Step 5. Calculate perc_diff between target and neighbour
    joined_gauges_gpcc_perc_diff = joined_gauges_gpcc.with_columns(
        perc_diff=calculate_perc_diff(
            pl.col(f"{rain_col}"), pl.col(f"{rain_col}_GPCC_{gpcc_id_name}")
        )
    )

    ## Step 6. Assign flags
    joined_gauges_gpcc_perc_flags = joined_gauges_gpcc_perc_diff.with_columns(
        pl.when((pl.col("perc_diff") <= -100.0))
        .then(-3)
        .when((pl.col("perc_diff") <= -50.0) & (pl.col("perc_diff") > -100.0))
        .then(-2)
        .when((pl.col("perc_diff") <= -25.0) & (pl.col("perc_diff") > -50.0))
        .then(-1)
        .when((pl.col("perc_diff") <= 25.0) & (pl.col("perc_diff") > -25.0))
        .then(0)
        .when((pl.col("perc_diff") >= 25.0) & (pl.col("perc_diff") < 50.0))
        .then(1)
        .when((pl.col("perc_diff") >= 50.0) & (pl.col("perc_diff") < 100.0))
        .then(2)
        .when((pl.col("perc_diff") >= 100.0))
        .then(3)
        .otherwise(0)
        .alias(f"monthly_flags_{gpcc_id_name}")
    )

    all_data = all_data.join(
        joined_gauges_gpcc_perc_flags[["time", f"monthly_flags_{gpcc_id_name}"]],
        on="time",
        how="left",
    )

    print(all_data[f"monthly_flags_{gpcc_id_name}"].value_counts())

shape: (8, 2)
┌────────────────────┬───────┐
│ monthly_flags_6303 ┆ count │
│ ---                ┆ ---   │
│ i32                ┆ u32   │
╞════════════════════╪═══════╡
│ 3                  ┆ 10    │
│ 0                  ┆ 16    │
│ 1                  ┆ 9     │
│ -2                 ┆ 1     │
│ null               ┆ 17    │
│ -1                 ┆ 2     │
│ -3                 ┆ 2     │
│ 2                  ┆ 3     │
└────────────────────┴───────┘
shape: (8, 2)
┌───────────────────┬───────┐
│ monthly_flags_310 ┆ count │
│ ---               ┆ ---   │
│ i32               ┆ u32   │
╞═══════════════════╪═══════╡
│ 3                 ┆ 11    │
│ 2                 ┆ 8     │
│ -3                ┆ 2     │
│ -1                ┆ 2     │
│ 1                 ┆ 10    │
│ 0                 ┆ 9     │
│ -2                ┆ 1     │
│ null              ┆ 17    │
└───────────────────┴───────┘
shape: (7, 2)
┌────────────────────┬───────┐
│ monthly_flags_3215 ┆ count │
│ ---                ┆ ---   │
│ i32      

In [137]:
all_data

time,rain_mm,monthly_flags_6303,monthly_flags_310,monthly_flags_3215
datetime[μs],f64,i32,i32,i32
2006-01-01 00:00:00,NaN,null,null,null
2006-02-01 00:00:00,NaN,null,null,null
2006-03-01 00:00:00,302.9,3,3,null
2006-04-01 00:00:00,224.2,3,3,null
2006-05-01 00:00:00,402.2,3,3,null
…,…,…,…,…
2010-08-01 00:00:00,417.1,3,2,3
2010-09-01 00:00:00,30.0,-2,-2,-2
2010-10-01 00:00:00,40.0,0,0,1


## Step 2. Calculate num neighbours online

In [138]:
## global determining the minimum number of non-null neighbours to count flags
MIN_NUM_NEIGHBOURS = 2  # original method is 3

In [139]:
num_neighbours = len(all_data.columns[2:])
num_neighbours

3

In [140]:
all_data_num_neighbours = all_data.with_columns(
    (
        num_neighbours
        - pl.sum_horizontal(pl.all().exclude("time").is_null().cast(pl.Int16))
    ).alias("num_neighbours_online")
)
all_data_num_neighbours

time,rain_mm,monthly_flags_6303,monthly_flags_310,monthly_flags_3215,num_neighbours_online
datetime[μs],f64,i32,i32,i32,i16
2006-01-01 00:00:00,NaN,null,null,null,0
2006-02-01 00:00:00,NaN,null,null,null,0
2006-03-01 00:00:00,302.9,3,3,null,2
2006-04-01 00:00:00,224.2,3,3,null,2
2006-05-01 00:00:00,402.2,3,3,null,2
…,…,…,…,…,…
2010-08-01 00:00:00,417.1,3,2,3,3
2010-09-01 00:00:00,30.0,-2,-2,-2,3
2010-10-01 00:00:00,40.0,0,0,1,3


In [141]:
all_data_num_neighbours["num_neighbours_online"].value_counts()

num_neighbours_online,count
i16,u32
0,17
3,33
2,10


In [142]:
all_data.filter(pl.col("monthly_flags_3215") == -3)

time,rain_mm,monthly_flags_6303,monthly_flags_310,monthly_flags_3215
datetime[μs],f64,i32,i32,i32
2009-06-01 00:00:00,0.0,-3,-3,-3
2009-07-01 00:00:00,0.0,-3,-3,-3


## Step 3. Majority voting flags for the target

In [143]:
all_data_monthly_flags = all_data_num_neighbours.with_columns(
    pl.when(pl.col("num_neighbours_online") < MIN_NUM_NEIGHBOURS)
    .then(np.nan)
    .otherwise(
        pl.min_horizontal(pl.all().exclude("time", rain_col, "num_neighbours_online"))
    )
    .alias("monthly_flags")
)

In [144]:
all_data_monthly_flags["monthly_flags"].value_counts()

monthly_flags,count
f64,u32
0.0,19
2.0,4
1.0,5
-1.0,3
-3.0,2
-2.0,1
NaN,17
3.0,9


In [145]:
all_data_monthly_flags.filter(
    (pl.col("monthly_flags") > 0) & (pl.col("monthly_flags").is_finite())
)

time,rain_mm,monthly_flags_6303,monthly_flags_310,monthly_flags_3215,num_neighbours_online,monthly_flags
datetime[μs],f64,i32,i32,i32,i16,f64
2006-03-01 00:00:00,302.9,3,3,null,2,3.0
2006-04-01 00:00:00,224.2,3,3,null,2,3.0
2006-05-01 00:00:00,402.2,3,3,null,2,3.0
2006-08-01 00:00:00,372.5,3,3,null,2,3.0
2006-09-01 00:00:00,89.1,3,3,null,2,3.0
…,…,…,…,…,…,…
2009-04-01 00:00:00,41.9,1,1,2,3,1.0
2009-11-01 00:00:00,1099.5,3,3,3,3,3.0
2010-08-01 00:00:00,417.1,3,2,3,3,2.0


## Step 4. Calculate neighbour monthly min/max climatology

In [158]:
all_gpcc_data = target_gauge_monthly.clone()

for gpcc_id_to_use, gpcc_path in existing_gpcc_monthly_paths.items():
    ## Step 0. Load GPCC monthly data
    gpcc_id_name = gpcc_id_to_use.split("DE_")[1].lstrip("0")
    if len(gpcc_path) == 0:
        continue
    f = zipfile.ZipFile(gpcc_path[0]).open(f"mw_{gpcc_id_name}.dat")
    example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

    ## Step 1. drop unnecessary columns
    example_gpcc = example_gpcc.drop([str(i) for i in range(3, 15)])

    ## Step 2. make datetime column (apparently it's 7am-7pm)
    example_gpcc = example_gpcc.with_columns(
        pl.datetime(year=pl.col("1"), month=pl.col("0"), day=1).alias("time")
    ).drop(["0", "1"])

    ## Step 3. rename and reorder
    example_gpcc = example_gpcc.rename({"2": f"{rain_col}_GPCC_{gpcc_id_name}"})
    example_gpcc = example_gpcc.select(
        ["time", f"{rain_col}_GPCC_{gpcc_id_name}"]
    )  ## Reorder (to look nice)

    ## Step 4. Add rows together climatology
    all_gpcc_data = all_gpcc_data.join(
        example_gpcc,
        on="time",
        how="left",
    )

In [201]:
all_gpcc_data_monthyear = make_monthyear_col(all_gpcc_data)

In [206]:
all_gpcc_data_month_min_max = all_gpcc_data_monthyear.group_by('month').agg([
        pl.min_horizontal(pl.all().exclude("time", "month", "year").min()).alias('neighbour_min'),
        pl.max_horizontal(pl.all().exclude("time", "month", "year").max()).alias('neighbour_max')
])

In [207]:
all_gpcc_data_month_min_max

month,neighbour_min,neighbour_max
i8,f64,f64
10,22.8,188.9
8,40.0,417.1
2,59.7,246.35
11,70.5,1099.5
5,43.3,402.2
…,…,…
3,54.7,302.9
12,51.5,160.25
1,34.0,226.3


## Step 5. Adjust flags to look for values outside of climatological range

In [237]:
all_data_monthly_flags = all_data_monthly_flags.select(['time', rain_col, 'num_neighbours_online', 'monthly_flags'])
all_data_monthly_flags_w_monthyear = make_monthyear_col(all_data_monthly_flags)

all_data_monthly_flags_w_climatology = all_data_monthly_flags_w_monthyear.join(all_gpcc_data_month_min_max, on='month')
all_data_monthly_flags_w_climatology = all_data_monthly_flags_w_climatology.drop("year", "month")

In [238]:
all_data_monthly_flags_w_climatology

time,rain_mm,num_neighbours_online,monthly_flags,neighbour_min,neighbour_max
datetime[μs],f64,i16,f64,f64,f64
2006-01-01 00:00:00,NaN,0,NaN,34.0,226.3
2006-02-01 00:00:00,NaN,0,NaN,59.7,246.35
2006-03-01 00:00:00,302.9,2,3.0,54.7,302.9
2006-04-01 00:00:00,224.2,2,3.0,1.0,4996.5
2006-05-01 00:00:00,402.2,2,3.0,43.3,402.2
…,…,…,…,…,…
2010-08-01 00:00:00,417.1,3,2.0,40.0,417.1
2010-09-01 00:00:00,30.0,3,-2.0,17.1,206.0
2010-10-01 00:00:00,40.0,3,0.0,22.8,188.9


In [239]:
## The intenseQC code is wrong here, so I am going off the description in the paper
all_data_monthly_flags_extreme = all_data_monthly_flags_w_climatology.with_columns(
    pl.when(
        (pl.col('monthly_flags') == -3) &
        (pl.col("num_neighbours_online") >= MIN_NUM_NEIGHBOURS) &
        (pl.col(rain_col) <= (0.75 * pl.col("neighbour_min"))) &
        (pl.col(rain_col) > (0.5 * pl.col("neighbour_min")))
    )
    .then(-4)
    .when(
        (pl.col('monthly_flags') == 3) &
        (pl.col("num_neighbours_online") >= MIN_NUM_NEIGHBOURS) &
        (pl.col(rain_col) < (0.5 * pl.col("neighbour_min")))
    )
    .then(-5)
    .when(
        (pl.col('monthly_flags') == 3) &
        (pl.col("num_neighbours_online") >= MIN_NUM_NEIGHBOURS) &
        (pl.col(rain_col) > (1.25 * pl.col("neighbour_max"))) &
        (pl.col(rain_col) < (2 * pl.col("neighbour_max")))
    )
    .then(4)
    .when(
        (pl.col('monthly_flags') == 3) &
        (pl.col("num_neighbours_online") >= MIN_NUM_NEIGHBOURS) &
        (pl.col(rain_col) > (2 * pl.col("neighbour_max")))
    )
    .then(5)
    .otherwise(pl.col("monthly_flags"))
    .alias("monthly_flags")

)

In [240]:
## NOTE: there are no -4, -5, 4 or 5 flags in this target data
all_data_monthly_flags_extreme['monthly_flags'].value_counts()

monthly_flags,count
f64,u32
NaN,17
-1.0,3
0.0,19
-2.0,1
-3.0,2
3.0,9
1.0,5
2.0,4


# QC21 - Timing offset 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [150]:
# # check for and remove any duplicates in neighbours list
# neighbours = list(dict.fromkeys(neighbours))

# # get gpcc
# neighbour_dfs = []
# for idx in neighbours:
#     neighbour_start_year = self.daily_dates[self.daily_names.index(idx)][0].year
#     neighbour_end_year = self.daily_dates[self.daily_names.index(idx)][1].year
#     neighbour_dfs.append(utils.get_daily_gpcc(self.daily_path, neighbour_start_year, neighbour_end_year, idx))

# # get matching stats for nearest gauge and offset
# nearest = neighbour_dfs[0].rename(columns={"GPCC": "ts2"})
# affinity_index_lag_minus1, r2_lag_minus1, _ = utils.calculate_affinity_index_and_pearson(daily_values_lag_minus1.to_frame("ts1"), nearest)
# affinity_index_lag0, r2_lag0, factor_lag0 = utils.calculate_affinity_index_and_pearson(daily_values.to_frame("ts1"), nearest)
# affinity_index_lag_plus1, r2_lag_plus1, _ = utils.calculate_affinity_index_and_pearson(daily_values_lag_plus1.to_frame("ts1"), nearest)

# affinity_indexes = [affinity_index_lag_minus1, affinity_index_lag0, affinity_index_lag_plus1]
# r2_values = [r2_lag_minus1, r2_lag0, r2_lag_plus1]

# if affinity_indexes.index(max(affinity_indexes)) == r2_values.index(max(r2_values)):
#     offset_flag = affinity_indexes.index(max(affinity_indexes)) - 1
# else:
#     offset_flag = 0

# QC22 - Pre-QC Affinity  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC23 - Pre-QC Pearson
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC24 - Daily factor  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [151]:
for n_id in all_neighbour_ids_paths.keys():
    print(n_id)
    one_neighbouring_gauge = get_neighbouring_gdsr_data(
        neighbour_gauge_id=n_id, time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
    )
    joined_gauges = target_gauge.join(
        one_neighbouring_gauge, on="time", suffix=f"_{n_id}"
    )
    joined_gauges = joined_gauges.drop_nans()

    ## resample into daily (also round to 1 decimal place) TODO: remove offset part
    joined_gauges = (
        joined_gauges.group_by_dynamic("time", every="1d", offset="7h")
        .agg(
            [
                pl.len().alias("n_hours"),
                pl.col(rain_col).mean().round(1).alias(rain_col),
                pl.col(f"{rain_col}_{n_id}")
                .mean()
                .round(1)
                .alias(f"{rain_col}_{n_id}"),
            ]
        )
        .filter(pl.col("n_hours") == 24)
        .drop("n_hours")
    )  # Ensure at least 24 data points

    ## NOTE: is this necessary? Why not just read resolution of each data?
    a = np.around(joined_gauges.filter(pl.col(rain_col) >= 0.1).min()[rain_col], 1)[0]
    b = np.around(
        joined_gauges.filter(pl.col(f"{rain_col}_{n_id}") >= 0.1).min()[
            f"{rain_col}_{n_id}"
        ],
        1,
    )[0]
    p = max(a, b, 0.1)
    print(a, b, p)

    ## TODO: rename all variables
    joined_gauges_duplicates = joined_gauges.with_columns(
        pl.when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_{n_id}") > p))
        .then(1)
        .when(
            (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_{n_id}") == p),
        )
        .then(1)
        .when(
            (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_{n_id}") > p),
        )
        .then(0)
        .when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_{n_id}") == p))
        .then(0)
        .otherwise(np.nan)
        .alias("duplicate")
    )

    joined_gauges = joined_gauges.with_columns(
        pl.when((pl.col(rain_col) > 0) & (pl.col(f"{rain_col}_{n_id}") > 0))
        .then(pl.col(rain_col) / pl.col(f"{rain_col}_{n_id}"))
        .otherwise(np.nan)
        .alias("factor")
    )

    match = (
        joined_gauges_duplicates["duplicate"]
        .value_counts()
        .filter(pl.col("duplicate") == 1)["count"]
        .item()
    )
    diff = (
        joined_gauges_duplicates["duplicate"]
        .value_counts()
        .filter(pl.col("duplicate") == 0)["count"]
        .item()
    )
    affinity = match / (match + diff)
    p_corr = np.corrcoef(joined_gauges[rain_col], joined_gauges[f"{rain_col}_{n_id}"])[
        0, 1
    ]
    f_mean = joined_gauges["factor"].drop_nans().mean()
    print(f"diff: {diff}, match:{match}")
    print(f"affinity: {affinity}, p_corr: {p_corr}, f_mean: {f_mean}")

    print("#" * 15)

DE_04313


TypeError: get_neighbouring_gdsr_data() got an unexpected keyword argument 'neighbour_gauge_id'. Did you mean 'neighbour_gdsr_id'?

# QC25 - Monthly factor
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 